In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",
            'random_class_idx': True,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': .01,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=MNIST/seed=10_epoch=1_lr=0.001_alpha=0.001_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[8 2 5 6 3 1 0 7 4 9]


In [3]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
metric_manager_callback.meters['accuracy'].data.shape

(6, 6, 1)

In [5]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [6]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 95.8125407892014, 'loss': 0.0005002836970721974, 'fairness': 1.7233935023796998}
training_task_end
---------------------------- Task 2 -----------------------
losses=tensor([[0.0475, 0.0174, 7.5142, 7.8053]])
### Cplex absolute_minsum LP solver ###
Elapsed time:7.414
Fairness:[ 0.10105552  0.31314608 -0.2156891  -0.19851232]
Current class loss:[ 0.          0.         -0.01025406 -0.00991053]
len(updated_seq_indices)=10000
[2] Eval metrics for task 1 >> {'accuracy': 75.33019355968355, 'loss': 0.0036637909331564177, 'fairness': 1.5108917116342768}
[2] Eval metrics for task 2 >> {'accuracy': 95.51557804469326, 'loss': 0.0006347872437657537, 'fairness': 1.4572820805677011}
training_task_end
---------------------------- Task 3 -----------------------
losses=tensor([[2.7578e-03, 1.7131e-03, 1.2940e-01, 8.6148e-02, 9.8699e+00, 8.1102e+00]])
### Cplex absolute_minsum LP solver ###
Elapsed t

In [7]:
metric_manager_callback.meters['fairness'].get_eer()

[0.0,
 10.092692242504853,
 13.073015096548668,
 16.699487058030634,
 14.422936160728108]

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[95.813,  0.   ,  0.   ,  0.   ,  0.   ],
       [75.33 , 95.516,  0.   ,  0.   ,  0.   ],
       [59.395, 82.097, 95.522,  0.   ,  0.   ],
       [56.911, 65.781, 92.259, 97.231,  0.   ],
       [54.119, 63.576, 89.421, 84.996, 92.269]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

83.03239835424907

In [10]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"fairness:{np.mean(metric_manager_callback.meters['fairness'].compute_overall())}")


accuracy:83.03239835424907
fairness:10.857626111562451
